In [1]:
import traci
import os
import sys
import keyboard
import time

# --- Configuration ---
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("Please declare environment variable 'SUMO_HOME'")

sumoBinary = "sumo-gui"  # Utilisez 'sumo-gui' pour voir la simulation
sumoCmd = [sumoBinary, "-c", "ff_heterogeneous.sumocfg", "--quit-on-end"]

In [2]:
def run_ref_simulation():
    traci.start(sumoCmd)
    
    # 1. On définit la route une seule fois au début
    traci.route.add("bus_route_2", ["E0E1", "E1E2", "E2E3", "E3E4", "E4E5"])
    
    bus_count = 0
    
    while traci.simulation.getTime() <= 3600:   # arrêter la simulation à 3600s
        traci.simulationStep() # Avancer d'une seconde (ou d'un pas)
        
        sim_time = traci.simulation.getTime()
        
        # 2. Vérifier si on doit ajouter un bus (toutes les 180s à partir de 180s)
        if sim_time >= 180 and sim_time % 180 == 0:
            bus_id = f"bus_2.{bus_count}"
            traci.vehicle.add(bus_id, "bus_route_2", typeID="bus")
            traci.vehicle.setVehicleClass(bus_id, "bus")
            print(f"Bus {bus_id} ajouté à t={sim_time}")
            bus_count += 1
            
        # Optionnel : Sortie manuelle avec la touche 'q'
        if keyboard.is_pressed('q'):
            break
            
    traci.close()

In [3]:
# --- Lancement ---
try:
    run_ref_simulation()
except traci.exceptions.FatalTraCIError:
    print("La simulation a été fermée.")

Bus bus_2.0 ajouté à t=180.0
Bus bus_2.1 ajouté à t=360.0
Bus bus_2.2 ajouté à t=540.0
Bus bus_2.3 ajouté à t=720.0
Bus bus_2.4 ajouté à t=900.0
Bus bus_2.5 ajouté à t=1080.0
Bus bus_2.6 ajouté à t=1260.0
Bus bus_2.7 ajouté à t=1440.0
Bus bus_2.8 ajouté à t=1620.0
Bus bus_2.9 ajouté à t=1800.0
Bus bus_2.10 ajouté à t=1980.0
Bus bus_2.11 ajouté à t=2160.0
Bus bus_2.12 ajouté à t=2340.0
Bus bus_2.13 ajouté à t=2520.0
Bus bus_2.14 ajouté à t=2700.0
Bus bus_2.15 ajouté à t=2880.0
Bus bus_2.16 ajouté à t=3060.0
Bus bus_2.17 ajouté à t=3240.0
Bus bus_2.18 ajouté à t=3420.0
Bus bus_2.19 ajouté à t=3600.0


**KPIs to compute**
- Mean travel time
- Time loss
- Arrival delay
- Mean waiting time
- Mean speed
- C02 emissions

In [4]:
import xml.etree.ElementTree as ET

def calculate_kpis(tripinfo_file):
    tree = ET.parse(tripinfo_file)
    root = tree.getroot()

    # Dictionnaires pour stocker les cumuls par mode
    # On initialise pour 'car' (voiture) et 'bus'
    stats = {
        'car': {'count': 0, 'duration': 0, 'timeLoss': 0, 'arrivalDelay': 0, 
                'waitingTime': 0, 'routeLength': 0, 'CO2': 0},
        'bus': {'count': 0, 'duration': 0, 'timeLoss': 0, 'arrivalDelay': 0, 
                'waitingTime': 0, 'routeLength': 0, 'CO2': 0}
    }

    for trip in root.findall('tripinfo'):
        v_type = trip.get('vType')
        
        # On vérifie si le type contient 'bus' ou 'car' 
        # (S'adapte si vos IDs sont 'bus.0', 'bus.1', etc.)
        mode = None
        if 'bus' in v_type.lower():
            mode = 'bus'
        elif 'car' in v_type.lower() or 'veh' in v_type.lower():
            mode = 'car'
            
        if mode:
            stats[mode]['count'] += 1
            stats[mode]['duration'] += float(trip.get('duration'))
            stats[mode]['timeLoss'] += float(trip.get('timeLoss'))
            stats[mode]['arrivalDelay'] += float(trip.get('arrivalDelay', 0))
            stats[mode]['waitingTime'] += float(trip.get('waitingTime'))
            stats[mode]['routeLength'] += float(trip.get('routeLength'))
            
            # Extraction des émissions CO2 depuis la balise enfant 'emissions'
            emissions = trip.find('emissions')
            if emissions is not None:
                stats[mode]['CO2'] += float(emissions.get('CO2_abs', 0))

    # Calcul des moyennes et affichage
    print(f"{'KPI':<25} | {'Voiture':<12} | {'Bus':<12}")
    print("-" * 55)
    
    for mode in ['car', 'bus']:
        n = stats[mode]['count']
        if n > 0:
            mean_travel_time = stats[mode]['duration'] / n
            mean_time_loss = stats[mode]['timeLoss'] / n
            mean_arrival_delay = stats[mode]['arrivalDelay'] / n
            mean_waiting_time = stats[mode]['waitingTime'] / n
            # Vitesse moyenne = distance totale / temps total
            mean_speed = stats[mode]['routeLength'] / stats[mode]['duration'] if stats[mode]['duration'] > 0 else 0
            mean_co2 = stats[mode]['CO2'] / n
            
            # Note : CO2_abs est en mg, on peut le convertir en g si besoin
            
            if mode == 'car':
                row_car = [mean_travel_time, mean_time_loss, mean_arrival_delay, 
                           mean_waiting_time, mean_speed, mean_co2]
            else:
                row_bus = [mean_travel_time, mean_time_loss, mean_arrival_delay, 
                           mean_waiting_time, mean_speed, mean_co2]

    labels = ["Mean Travel Time (s)", "Time Loss (s)", "Arrival Delay (s)", 
              "Mean Waiting Time (s)", "Mean Speed (m/s)", "CO2 Emissions (mg)"]
    
    for i in range(len(labels)):
        print(f"{labels[i]:<25} | {row_car[i]:<12.2f} | {row_bus[i]:<12.2f}")

# Utilisation
# calculate_kpis('results_tripinfo.xml')

In [5]:
calculate_kpis('results_tripinfo.xml')

KPI                       | Voiture      | Bus         
-------------------------------------------------------
Mean Travel Time (s)      | 356.81       | 359.06      
Time Loss (s)             | 288.93       | 287.50      
Arrival Delay (s)         | 0.00         | 0.00        
Mean Waiting Time (s)     | 238.16       | 240.17      
Mean Speed (m/s)          | 2.46         | 2.75        
CO2 Emissions (mg)        | 0.00         | 0.00        
